In [ ]:
import requests
import os
import pandas as pd
import numpy as np
import sqlite3
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
data_url1 = 'https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/EDGAR/datasets/v61_AP/NMVOC/v61_AP_NMVOC_1970_2018b.zip'
data_name1 = 'emissions'

In [ ]:
data_url2 = 'https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/GWA02/CSV/1.0/en'
data_name2 = 'treat_waste'

In [ ]:
data_url3 = 'https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/GWA01/CSV/1.0/en'
data_name3 = 'generate_waste'

In [ ]:
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Directory '{data_dir}' created.")

In [ ]:
df_xls=None

with urlopen(data_url1) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        for file in zfile.namelist():
            if not file.endswith('_readme.html'):
#                 zfile.extract(member=file, path=data_dir)
#                 os.rename(os.path.join(data_dir, file), os.path.join(data_dir, 'NMVOC_emissions.xlsx')) 
#                 print(f"File '{file}' extracted and renamed.")
                with zfile.open(file) as excel_file_content:
                       df_xls = pd.read_excel(excel_file_content,skiprows=9)
            
print('zip file imported and created draframe from xlsx file')
df_xls.head()


In [ ]:
columns_to_keep = ['ipcc_code_2006_for_standard_report_name'] + [f'Y_{year}' for year in range(2004, 2019)]
columns_to_drop = ['Name','IPCC_annex', 'C_group_IM24_sh', 'Country_code_A3', 'ipcc_code_2006_for_standard_report', 'Substance', 'fossil_bio']

# Filter and rename columns
IE_rows = df_xls.drop(columns=columns_to_drop).loc[df_xls['Name'] == 'Ireland', columns_to_keep].rename(
    columns={'ipcc_code_2006_for_standard_report_name': 'Emission Sector'}
)
IE_rows.dropna(inplace=True)
IE_rows.reset_index(inplace=True, drop =True)

IE_rows.head()

In [ ]:
# # for col in IE_rows.columns:
# #     print(f"Unique values in {col}: {IE_rows[col].unique()}")
# unique_emission_sectors_df = pd.DataFrame(IE_rows, columns=['Emission Sector'])
# unique_emission_sectors_df

In [ ]:
db_full_path = os.path.join(data_dir, data_name1)
db_con1 = sqlite3.connect(db_full_path)
IE_rows.to_sql('emissions', db_con1, if_exists='replace', index=False)
# db_con1.commit()
db_con1.close()

In [ ]:
df_treat = pd.read_csv(data_url2)
df_treat.reset_index(inplace=True)

columns_to_drop = ['index','STATISTIC', 'Statistic Label', 'TLIST(A1)', 'C04253V05027', 'C04251V05025','C04252V05026','UNIT']

df_treat = df_treat.drop(columns=columns_to_drop)
df_treat.dropna(inplace=True)
# Remove rows with 'total waste' in the 'Waste Category' column
df_treat = df_treat[df_treat['Waste Category'] != 'Total waste']
df_treat.reset_index(inplace=True, drop =True)
df_treat.head()

db_full_path = os.path.join(data_dir, data_name2)
db_con2 = sqlite3.connect(db_full_path)
df_treat.to_sql('treatment', db_con2, if_exists='replace', index=False)
# db_con3.commit()
db_con2.close()

In [ ]:
df_gen = pd.read_csv(data_url3)
df_gen.reset_index(inplace=True)

columns_to_drop = ['index','STATISTIC', 'Statistic Label', 'TLIST(A1)', 'C014259V05033', 'C04253V05027','C04251V05025','UNIT']

df_gen = df_gen.drop(columns=columns_to_drop)
df_gen.dropna(inplace=True)

# Remove rows with 'total waste' in the 'Waste Category' column
df_gen = df_gen[df_gen['Waste Category'] != 'Total waste']
df_gen.reset_index(inplace=True, drop =True)
df_gen.head()

db_full_path = os.path.join(data_dir, data_name3)
db_con3 = sqlite3.connect(db_full_path)
df_gen.to_sql('generate', db_con3, if_exists='replace', index=False)
# db_con3.commit()
db_con3.close()

In [ ]:
# Calculate the total waste generated for each year
total_waste_gen_by_year = df_gen.groupby('Year')['VALUE'].sum()
# Calculate the total waste treated for each year
total_waste_treat_by_year = df_treat.groupby('Year')['VALUE'].sum()

waste_emissions_df = IE_rows[IE_rows['Emission Sector'].isin(['Solid Waste Disposal', 
                                                             'Incineration and Open Burning of Waste', 
                                                             'Biological Treatment of Solid Waste'])]
# Now, let's calculate the total waste emissions for each year
waste_emissions_by_year = waste_emissions_df.iloc[:, 1:].sum()
# Now, let's calculate the total emissions for all sectors for each year
total_emissions_by_year = IE_rows.iloc[:, 1:].sum()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

#trend of total waste generation over time
axes[0].plot(total_waste_gen_by_year.index, total_waste_gen_by_year.values, marker='o', linestyle='-', label='Generation')
axes[0].plot(total_waste_treat_by_year.index, total_waste_treat_by_year.values, marker='o', linestyle='-', color='red', label='Treatment')
axes[0].set_title('Comparison of Total Waste Generation and Treatment Over Time')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Total Waste (t)')
axes[0].legend()
axes[0].grid(True)

#trend of total emissions and waste emissions over time
axes[1].plot(total_emissions_by_year.index, total_emissions_by_year.values, marker='o', linestyle='-', label='Total Emissions')
axes[1].plot(waste_emissions_by_year.index, waste_emissions_by_year.values, marker='o', linestyle='-', label='Waste Sector Emissions')
axes[1].set_title('Trend of Total (NMVOC) Emissions and Waste Sector Emissions Over Time')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Emissions (t)')
axes[1].legend()
axes[1].grid(True)
axes[1].set_xticks(total_emissions_by_year.index)
axes[1].tick_params(axis='x', rotation=45)